In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Interactive RCL temperature data

View Ruapehu Crater Lake temperature data interactively.

This is a test using PyViz

[<img src="https://raw.githubusercontent.com/pyviz/pyviz/master/notebooks/assets/PyViz_logo_wm.png" width="42" align="left">](http://pyviz.org)

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh', logo=False)

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
names = ['dt', 'temp', 'err']

In [ ]:
df = pd.read_csv('https://fits.geonet.org.nz/observation?siteID=RU001&typeID=t', skiprows=1, names=names, parse_dates=['dt'], index_col='dt')
#resample as daily mean and interpolate
dfd = df.resample('D').mean().interpolate(method='linear')

In [ ]:
#holoviews requires a non-datetime index
dfday = dfd.reset_index()

In [ ]:
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[
    ("Date", "@dt{%Y-%m-%d}"),
    ("Temperature", "@temp")
], formatters={"dt":"datetime"}, mode='vline')

In [ ]:
%%opts Curve  [height=600 width=1200 tools=[hover] title_format='Ruapehu Crater Lake daily mean temperature']

#horizontal lines for descrptive statistics
median = hv.HLine(dfd['temp'].median()).options(color='green', line_width=2, line_dash='dashed', alpha=0.5)
q25 = hv.HLine(dfd['temp'].quantile(q=0.25)).options(color='black', line_width=2, line_dash='dashed', alpha=0.5)
q75 = hv.HLine(dfd['temp'].quantile(q=0.75)).options(color='red', line_width=2, line_dash='dashed', alpha=0.5)

#label horizontal lines
median_label = hv.Text(pd.to_datetime('1951-01-01'), dfd['temp'].median()*1.025, "median", halign='left').options(color='green', text_alpha=0.5)
q25_label = hv.Text(pd.to_datetime('1951-01-01'), dfd['temp'].quantile(q=0.25)*1.025, "lower quartile", halign='left').options(color='black', text_alpha=0.5)
q75_label = hv.Text(pd.to_datetime('1951-01-01'), dfd['temp'].quantile(q=0.75)*1.025, "upper quartile", halign='left').options(color='red', text_alpha=0.5)

curve = hv.Curve(dfday, 'dt', 'temp').redim.label(dt=' ',temp='Temperature (C)') * median * q25 * q75  * median_label * q25_label * q75_label
curve